In [ ]:
from cvxopt import matrix
from cvxopt import solvers
import numpy as np

In [ ]:
def kernel_cal(X, Y, choose, gama=0.1):
    '''
    Args:
        X: m1 x n
        Y: m2 x n
        choose: choosen kernel, 'linear' or 'RBF'
        gama: parameter for Guassian kernel, default 0.1
    Returns:
        K: m1 x m2
    '''
    if choose == 'linear':
        K = np.inner(X, Y)
    elif choose == 'RBF': 
        K = np.ones((X.shape[0], Y.shape[0]))
        for i in range(X.shape[0]):
            for j in range(Y.shape[0]):
                K[i, j] = np.exp(-gama * np.linalg.norm(X[i, :] - Y[j, :]) ** 2)
    else:
        print('please choose the kernel')
    return K

In [ ]:
def SVM_cal(X, Y, C, kernel, gama=0.01, pick=0.01):
    '''
    Args:
        X: training data, n x 3
        Y: training label, n x 1
        C: soft scale
        kernel: choosed kernel type, 'linear' or 'RBF'
        pick: alpha > pick, x is SV
    Returns:
        (sv_x, sv_y): support vector, (n x 2, n x 1)
        sv_alpha: pre_coefficients, n x 1
    '''
    # alpha n x 1
    n_sample = X.shape[0]
    x_kernel = kernel_cal(X, X, kernel, gama)
    y_out = np.outer(Y, Y) # n x n
    P = matrix(y_out * x_kernel) # n x n
    Q = matrix(-1 * np.ones((n_sample, 1))) # n x 1
    b = matrix(0.0)
    A = matrix(Y.transpose()) # 1 x n
    G_C = matrix(np.diag(np.ones(n_sample)))
    G_0 = matrix(np.diag(-1 * np.ones(n_sample)))
    h_C = matrix(C * np.ones((n_sample, 1)))
    h_0 = matrix(np.zeros((n_sample, 1)))
    G = matrix(np.vstack((G_C, G_0))) # 2n x n
    h = matrix(np.vstack((h_C, h_0))) # 2n x 1
    result = solvers.qp(P, Q, G, h, A, b)
    solvers.options['show_progress'] = False
    # calculate alpha
    alpha = result['x']
    alpha = np.asarray(alpha) # n x 1
    # choose SV
    loc = np.where(alpha > pick)
    sv_x = X[loc, 0: 2]
    sv_y = Y[loc, :]
    sv_alpha = alpha[loc, :]
    SVM_dict = {'sv_x': sv_x[0, :, :], 'sv_y': sv_y[0, :, :], 'sv_alpha': sv_alpha[0, :, :], 'kernel': kernel, 'gama': gama}
    return SVM_dict